In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm
import cv2
import tifffile
from skimage.transform import resize
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, Reshape, Input, Conv2DTranspose
from keras.layers import Activation, LeakyReLU, BatchNormalization, Dropout, Resizing
try:
    from tensorflow.keras.optimizers import Adam
except:
    from keras.optimizers import Adam

In [ ]:
NOISE_DIM = 100  
BATCH_SIZE = 20 
STEPS_PER_EPOCH = 100
EPOCHS = 100
SEED = 40
WIDTH, HEIGHT, CHANNELS = 75, 75, 2

OPTIMIZER = Adam(0.0002, 0.5)

In [ ]:
train = pd.read_json('./data/train.json')
train['inc_angle'] = pd.to_numeric(train['inc_angle'],errors='coerce')

In [ ]:
def get_stats(train,label=1):
    train['max'+str(label)] = [np.max(np.array(x)) for x in train['band_'+str(label)] ]
    train['maxpos'+str(label)] = [np.argmax(np.array(x)) for x in train['band_'+str(label)] ]
    train['min'+str(label)] = [np.min(np.array(x)) for x in train['band_'+str(label)] ]
    train['minpos'+str(label)] = [np.argmin(np.array(x)) for x in train['band_'+str(label)] ]
    train['med'+str(label)] = [np.median(np.array(x)) for x in train['band_'+str(label)] ]
    train['std'+str(label)] = [np.std(np.array(x)) for x in train['band_'+str(label)] ]
    train['mean'+str(label)] = [np.mean(np.array(x)) for x in train['band_'+str(label)] ]
    train['p25_'+str(label)] = [np.sort(np.array(x))[int(0.25*75*75)] for x in train['band_'+str(label)] ]
    train['p75_'+str(label)] = [np.sort(np.array(x))[int(0.75*75*75)] for x in train['band_'+str(label)] ]
    train['mid50_'+str(label)] = train['p75_'+str(label)]-train['p25_'+str(label)]

    return train
train = get_stats(train,1)
train = get_stats(train,2)

In [ ]:
def plot_var(name,nbins=50):
    minval = train[name].min()
    maxval = train[name].max()
    plt.hist(train.loc[train.is_iceberg==1,name],range=[minval,maxval],
             bins=nbins,color='b',alpha=0.5,label='Boat')
    plt.hist(train.loc[train.is_iceberg==0,name],range=[minval,maxval],
             bins=nbins,color='r',alpha=0.5,label='Iceberg')
    plt.legend()
    plt.xlim([minval,maxval])
    plt.xlabel(name)
    plt.ylabel('Number')
    plt.show()

In [ ]:
for col in ['inc_angle','min1','max1','std1','med1','mean1','mid50_1']:
    plot_var(col)

In [ ]:
train_stats = train.drop(['id','is_iceberg','band_1','band_2'],axis=1)


In [ ]:
corr = train_stats.corr()
fig = plt.figure(1, figsize=(10,10))
plt.imshow(corr,cmap='inferno')
labels = np.arange(len(train_stats.columns))
plt.xticks(labels,train_stats.columns,rotation=90)
plt.yticks(labels,train_stats.columns)
plt.title('Correlation Matrix of Global Variables')
cbar = plt.colorbar(shrink=0.85,pad=0.02)
plt.show()

In [ ]:
icebergs = train[train.is_iceberg==1].sample(n=300,random_state=123)
ships = train[train.is_iceberg==0].sample(n=300,random_state=456)


In [ ]:
# Plot band_1/band_2 images
fig = plt.figure(1,figsize=(15,15))
for i in range(9):
    ax = fig.add_subplot(3,3,i+1)
    arr = np.reshape(np.array(icebergs.iloc[i,1]),(75,75))
    ax.imshow(arr,cmap='inferno')
    
plt.show()

In [ ]:
def plot_results(images, n_cols=None):
    '''Visualizes fake images'''

    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1

    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)

    plt.figure(figsize=(n_cols * 2, n_rows * 2))  # Increase the figure size to accommodate 2-band images

    for index, image in enumerate(images):
        if image.shape[-1] == 1:
            image = np.squeeze(image, axis=-1)
        elif image.shape[-1] == 2:
            # Merge the two bands into a single image
            image = np.dstack((image[:, :, 0], image[:, :, 1]))
            image = np.mean(image, axis=2)  # Convert to grayscale by taking the mean of the two bands
        else:
            raise ValueError("Invalid number of image bands")

        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="inferno")
        plt.axis("off")


In [ ]:
# download the training images
X_train = icebergs[['band_1', 'band_2']]
# Remove the first row (column names)
column_names = X_train.iloc[0]
X_train = X_train.iloc[1:]

column_1 = np.array([np.array(image) for image in X_train['band_1']])
column_2 = np.array([np.array(image) for image in X_train['band_2']])

column_1 = column_1.flatten().reshape(-1)
column_2 = column_2.flatten().reshape(-1)

mean_1 = np.mean(column_1)
std_1 = np.std(column_1)

mean_2 = np.mean(column_2)
std_2 = np.std(column_2)

normalized_column_1 = (column_1 - column_1.min()) / (column_1.max() - column_1.min())
normalized_column_2 = (column_2 - column_2.min()) / (column_2.max() - column_2.min())
#normalized_column_1 = (column_1 - mean_1) / std_1
#normalized_column_2 = (column_2 - mean_2) / std_2

X_train_normalized = np.column_stack((normalized_column_1, normalized_column_2))
#print(X_train[0])
# Reshape images 
X_train = X_train_normalized.reshape(-1, WIDTH,HEIGHT,CHANNELS)

# Convert X_train to a NumPy array
X_train_array = np.array(X_train)


print(X_train_array.shape)

# create batches of tensors to be fed into the model
dataset = X_train
print(X_train_array.dtype)

In [ ]:
def build_generator_dcgan():
    model = Sequential([
        Dense(25 * 25 * 256, input_dim=NOISE_DIM),
        LeakyReLU(alpha=0.2),
        Reshape((25, 25, 256)),
        
        Conv2DTranspose(32, (2, 2), strides=(3, 3), padding='valid'),
        LeakyReLU(alpha=0.2),

        Conv2DTranspose(64, (2, 2), strides=(1, 1), padding='valid'),
        LeakyReLU(alpha=0.2),
        
        Conv2DTranspose(128, (2, 2), strides=(1, 1), padding='valid'),
        LeakyReLU(alpha=0.2),

        Conv2D(2, (3, 3), padding='valid', activation='tanh')
    ], name="generator")

    model.summary()
    model.compile(loss="binary_crossentropy", optimizer=OPTIMIZER)

    return model


In [ ]:
def build_discriminator_dcgan():
    model = Sequential([
        Conv2D(64, (3, 3), padding='same', input_shape=(75, 75, 2)),
        LeakyReLU(alpha=0.2),

        Conv2D(75, (3, 3), strides=2, padding='same'),
        LeakyReLU(alpha=0.2),

        Conv2D(75, (3, 3), strides=2, padding='same'),
        LeakyReLU(alpha=0.2),

        Conv2D(75, (3, 3), strides=2, padding='same'),
        LeakyReLU(alpha=0.2),

        Flatten(),
        Dropout(0.4),
        Dense(1, activation="sigmoid")
    ], name="discriminator")
    
    model.summary()
    model.compile(loss="binary_crossentropy", optimizer=OPTIMIZER)
    
    return model


In [ ]:
print('\n')
discriminator_dcgan = build_discriminator_dcgan()
print('\n')
generator_dcgan = build_generator_dcgan()
trainable_discriminator_vars = discriminator_dcgan.trainable_variables
trainable_generator_vars = generator_dcgan.trainable_variables
trainable_vars = trainable_discriminator_vars + trainable_generator_vars
OPTIMIZER.build(trainable_vars)

discriminator_dcgan.trainable = False 

gan_input = Input(shape=(NOISE_DIM,))
fake_image = generator_dcgan(gan_input)
print(fake_image.shape)
dcgan_output = discriminator_dcgan(fake_image)

dcgan = Model(gan_input, dcgan_output, name="gan_model")
dcgan.compile(loss="binary_crossentropy", optimizer=OPTIMIZER)

#print("The Combined Network:\n")
#gan.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# Generator Model
def build_generator_wgan():
    model = tf.keras.Sequential()
    model.add(layers.Dense(18 * 18 * 256, use_bias=False, input_shape=(75 * 75 * 2,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((18, 18, 256)))

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(2, (4, 4), strides=(1, 1), padding='valid', use_bias=False, activation='tanh'))

    return model


# Critic Model
def build_critic_wgan():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[75, 75, 2]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

# Define the loss function for the critic
def critic_loss(real_output, fake_output):
    return tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)

# Define the loss function for the generator
def generator_loss(fake_output):
    return -tf.reduce_mean(fake_output)

# Gradient penalty function
def gradient_penalty(critic, real_images, fake_images):
    BATCH_SIZE = tf.shape(real_images)[0]
    alpha = tf.random.uniform(shape=[BATCH_SIZE, 1, 1, 1], minval=0.0, maxval=1.0)
    real_images = tf.cast(real_images, tf.float32)
    interpolated_images = real_images[:BATCH_SIZE] + alpha * (fake_images[:BATCH_SIZE] - real_images[:BATCH_SIZE])

    with tf.GradientTape() as tape:
        tape.watch(interpolated_images)
        critic_interpolated = critic(interpolated_images)

    gradients = tape.gradient(critic_interpolated, interpolated_images)
    gradients_norm = tf.norm(gradients)
    gradient_penalty = tf.reduce_mean((gradients_norm - 1.0) ** 2)

    return gradient_penalty


# Training loop
def train_wgan(images, epochs=100, batch_size=64, critic_steps=5):
    # Normalize the input images to the range [-1, 1]
    images = (images - 0.5) * 2.0

    generator = build_generator_wgan()
    critic = build_critic_wgan()

    # Define the optimizers for the generator and critic
    generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    critic_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

    @tf.function
    def train_step(images):
        for i in range(critic_steps):
            # Generate random noise as input to the generator
            noise = tf.random.normal([batch_size, 75*75*2])
            print('critic range', i)
            with tf.GradientTape() as critic_tape:
                # Generate fake images from the noise using the generator
                generated_images = generator(noise)

                # Get the critic's output for real and fake images
                real_output = critic(images)
                fake_output = critic(generated_images)

                # Compute the critic loss and the gradient penalty
                critic_loss_value = critic_loss(real_output, fake_output)
                gp = gradient_penalty(critic, images, generated_images)
                total_loss = critic_loss_value + 10.0 * gp

            # Compute the gradients and update the critic's parameters
            critic_gradients = critic_tape.gradient(total_loss, critic.trainable_variables)
            critic_optimizer.apply_gradients(zip(critic_gradients, critic.trainable_variables))

        # Generate random noise as input to the generator
        noise = tf.random.normal([batch_size, 75*75*2])

        with tf.GradientTape() as generator_tape:
            # Generate fake images from the noise using the generator
            generated_images = generator(noise)

            # Get the critic's output for the generated images
            fake_output = critic(generated_images)

            # Compute the generator loss
            generator_loss_value = generator_loss(fake_output)

        # Compute the gradients and update the generator's parameters
        generator_gradients = generator_tape.gradient(generator_loss_value, generator.trainable_variables)
        generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))

        return critic_loss_value, generator_loss_value

    # Create a dataset from the input images
    dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(len(images)).batch(batch_size)

    # Training loop
    for epoch in range(epochs):
        print(epoch)
        for image_batch in dataset:
            critic_loss_value, generator_loss_value = train_step(image_batch)

        # Print the losses for monitoring the training progress
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Critic Loss: {critic_loss_value:.4f}, Generator Loss: {generator_loss_value:.4f}")

    return generator

# Example usage:
# Assuming you have an array of images called 'input_images' with shape (num_images, 75, 75, 2)
input_images = X_train
print(input_images.shape)
generator_wgan = train_wgan(input_images, epochs=5, batch_size=32, critic_steps=5)

In [138]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Generator model
def build_generator_pix2pix():
    inputs = tf.keras.Input(shape=(75, 75, 2))

    # Encoder
    down1 = layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same')(inputs)
    down2 = layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same')(layers.LeakyReLU()(down1))
    
    # Decoder
    up1 = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(layers.LeakyReLU()(down2))
    up2 = layers.Conv2DTranspose(2, (3, 3), strides=(2, 2), padding='same')(layers.LeakyReLU()(up1))

    # Output
    outputs = layers.Activation('tanh')(up2)
    return models.Model(inputs=inputs, outputs=outputs)

# Discriminator model
def build_discriminator_pix2pix():
    inputs = tf.keras.Input(shape=(75, 75, 4))

    # Convolutional layers
    conv1 = layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same')(inputs)
    conv2 = layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same')(layers.LeakyReLU()(conv1))
    conv3 = layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same')(layers.LeakyReLU()(conv2))
    
    # Classification layer
    outputs = layers.Conv2D(1, (3, 3), strides=(1, 1), padding='same')(layers.LeakyReLU()(conv3))

    return models.Model(inputs=inputs, outputs=outputs)

# Define the loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Discriminator loss function
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# Generator loss function
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers for generator and discriminator
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Initialize generator and discriminator models
generator = build_generator_pix2pix()
discriminator = build_discriminator_pix2pix()
# Training loop
@tf.function
def train_step(input_images, target_images):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake images
        generated_images = generator(X_train_array, training=True)
        print('oxe')
        # Discriminator loss
        real_output = discriminator(tf.concat([input_images, target_images], axis=-1), training=True)
        fake_output = discriminator(tf.concat([input_images, generated_images], axis=-1), training=True)
        disc_loss = discriminator_loss(real_output, fake_output)

        # Generator loss
        gen_loss = generator_loss(fake_output)

    # Calculate gradients and apply them
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Training loop
def train(dataset, epochs):
    for epoch in range(epochs):
        
        for input_images, target_images in dataset:
            train_step(input_images, target_images)


# Define a function to split the paired images into input and target images
def split_images(real_image, target_image):
    return real_image, target_image

# Assuming X_train_array has shape (num_samples, 75, 75, 2)
num_samples = X_train_array.shape[0]
# Generate random noise as input images
input_images = np.random.normal(size=(num_samples, 75, 75, 2))

# Create dataset with input and target images
dataset = tf.data.Dataset.from_tensor_slices((input_images, X_train_array))
train(dataset, epochs=100)


(75, 75, 2)
(75, 75, 2)
oxe


ValueError: in user code:

    File "C:\Users\flopes1\AppData\Local\Temp\ipykernel_52376\4003475053.py", line 63, in train_step  *
        real_output = discriminator(tf.concat([input_images, target_images], axis=-1), training=True)
    File "C:\Users\flopes1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\flopes1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "model_47" is incompatible with the layer: expected shape=(None, 75, 75, 4), found shape=(75, 75, 4)


In [ ]:
def sample_images(noise, subplots, generator, prefix, figsize=(22,8), save=False):
    generated_images = generator.predict(noise)
    plt.figure(figsize=figsize)
    #print(np.amax(generated_images))
    for i, image in enumerate(generated_images):
        plt.subplot(subplots[0], subplots[1], i+1)
        if CHANNELS == 1:
            plt.imshow(image.reshape((WIDTH, HEIGHT)), cmap='gray')    
                                                                            
        else:
            image_band1 = np.reshape(np.array(image[:, :, 1]),(75,75))
            plt.imshow(image_band1, cmap='inferno')

            #channel_1 = image[:, :, 0]
            #channel_2 = image[:, :, 1]

            #plt.subplot(1, 2, 1)
            #plt.imshow(channel_1, cmap='gray')
            #plt.title('Channel 1')

            #plt.subplot(1, 2, 2)
            #plt.imshow(channel_2, cmap='gray')
            #plt.title('Channel 2')

            #plt.show()
            import matplotlib.image
            #print(np.amin(image.reshape((WIDTH, HEIGHT, 3))))
            from skimage import util
            #image = (image - image.min()) / (image.max() - image.min())
            #print(np.min(image))   # minimum value of the image data
            #print(np.max(image))   # maximum value of the image data
            #matplotlib.image.imsave('gen_img'+str(i)+'.png', image)
            #image_data = np.transpose(image, (2, 0, 1))
            # Transpose the array to have shape (height, width, num_channels)
            #image_data = np.transpose(image, (2, 0, 1))
            #print(image.shape)
            # Concatenate the channels along the third axis to create a multi-channel image
            print(image.shape)
            joined_channels_image = np.concatenate([np.expand_dims(image[:, :, i], axis=0) for i in range(image.shape[2])], axis=0)
            plt.imshow(joined_channels_image)
            # Save grayscale image as PNG
            plt.imsave(prefix + '_gen_img' + str(i) + '.png', joined_channels_image, cmap='inferno')

        if save == True:
            img_name = "gen" + str(i)
            plt.savefig(img_name)
        plt.subplots_adjust(wspace=None, hspace=None)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
np.random.seed(SEED)
for epoch in range(100):
    
    for batch in tqdm(range(10)):

        noise = np.random.normal(0,1, size=(BATCH_SIZE, NOISE_DIM))
        fake_X = generator_dcgan.predict(noise)
        
        idx = np.random.randint(0, X_train_array.shape[0], size=BATCH_SIZE)
        real_X = X_train_array[idx]
        X = np.concatenate((real_X, fake_X))

        disc_y = np.zeros(2*BATCH_SIZE)
        disc_y[:BATCH_SIZE] = 1

        d_loss = discriminator_dcgan.train_on_batch(X, disc_y)
        
        y_gen = np.ones(BATCH_SIZE)
        g_loss = dcgan.train_on_batch(noise, y_gen)

    print(f"EPOCH: {epoch + 1} Generator Loss: {g_loss:.4f} Discriminator Loss: {d_loss:.4f}")
    noise = np.random.normal(0, 1, size=(BATCH_SIZE,NOISE_DIM))
    #sample_images(noise, (2,5))

In [ ]:
#noise = np.random.normal(0, 1, size=(100, NOISE_DIM))
noise = tf.random.normal([32, 75*75*2])
sample_images(noise, (10,10), generator_dcgan, 'dcgan', (24,20), save=True)
sample_images(noise, (10,10), generator_wgan, 'wgan', (24,20), save=True)
sample_images(noise, (10,10), generator_pix2pix, 'pix2pix', (24,20), save=True)